![title](edel.JPG)

# Edelweiss Hackathon-Machine Learning

## Predict Forclosure Probability

### Problem Statement

India is a diversified financial sector undergoing rapid expansion, both in terms of strong growth of existing financial services firms and new entities entering the market. The sector comprises commercial banks, insurance companies, non-banking financial companies, co-operatives, pension funds, mutual funds and other smaller financial entities .
The Edelweiss Group is one of India&#39;s leading diversified financial services company providing a broad range of financial products and services to a substantial and diversified client base that includes corporations, institutions and individuals. Edelweiss&#39;s products and services span multiple asset classes and consumer segments across domestic and global geographies.
Given the availability of various alternatives across the industry customer has a propensity to move to another financial institution for Balance Transfer. Foreclosure and balance transfer has added to the concerns of NBFCs. Foreclosure means repaying the outstanding loan amount in a single payment instead of with EMIs while balance transfer is transferring outstanding Loan availed from one Bank / Financial Institution to another Bank / Financial Institution, usually on the grounds of better service, top-up on the existing loan, proximity of branch, saving on interest repayments, etc. Losing out on customers on grounds on foreclosure and balance transfer leads to revenue loss. Acquiring a new customer can cost up to five times more than retaining an existing customer and an increase in customer retention by 5% increases profits up to 25%.
NBFCs have started taking pro-active measures to ensure this is curbed; and this is where you come in! Objective is primarily to arrive at a propensity to foreclose and balance transfer an existing loan based on lead indicators such as demographics, internal behavior and performance on all credit lines; along with the estimated ‘Time to Foreclose’. May the best algorithm win!

Participants are free to use any openly available data source to further enrich the dataset.


### Data Description
We have provided the following dataset:
* **Customer demographics**
* **Customer transactions (ie. repayments made by the client)**
* **List of foreclosed customers **
* **Email interaction of the customer with the customer representative**

The participants are expected to predict the probability of foreclosure for each of the data points in the test set.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input/foreclosure-prediction-dataset"))

# Any results you write to the current directory are saved as output.

['test_foreclosure.csv', 'RF_Final_Data.xlsx', 'LMS_31JAN2019.xlsx', 'Customers_31JAN2019.xlsx', 'train_foreclosure.csv']


In [2]:
train=pd.read_csv('../input/foreclosure-prediction-dataset/train_foreclosure.csv')
test=pd.read_csv('../input/foreclosure-prediction-dataset/test_foreclosure.csv')
final=pd.read_excel('../input/foreclosure-prediction-dataset/RF_Final_Data.xlsx')
tran1=pd.read_excel('../input/foreclosure-prediction-dataset/LMS_31JAN2019.xlsx')
cust=pd.read_excel('../input/foreclosure-prediction-dataset/Customers_31JAN2019.xlsx')
tran1.head()

,AGREEMENTID,CUSTOMERID,LOAN_AMT,NET_DISBURSED_AMT,INTEREST_START_DATE,CURRENT_ROI,ORIGNAL_ROI,CURRENT_TENOR,ORIGNAL_TENOR,DUEDAY,AUTHORIZATIONDATE,CITY,PRE_EMI_DUEAMT,PRE_EMI_RECEIVED_AMT,PRE_EMI_OS_AMOUNT,EMI_DUEAMT,EMI_RECEIVED_AMT,EMI_OS_AMOUNT,EXCESS_AVAILABLE,EXCESS_ADJUSTED_AMT,BALANCE_EXCESS,NET_RECEIVABLE,OUTSTANDING_PRINCIPAL,PAID_PRINCIPAL,PAID_INTEREST,MONTHOPENING,LAST_RECEIPT_DATE,LAST_RECEIPT_AMOUNT,NET_LTV,COMPLETED_TENURE,BALANCE_TENURE,DPD,FOIR,PRODUCT,SCHEMEID,NPA_IN_LAST_MONTH,NPA_IN_CURRENT_MONTH,MOB
0,11220001,12001000.0,1.171011e+07,1.171011e+07,2010-08-30,13.734072,10.612692,58.0,120,1,2010-08-29,MUMBAI,7994.273589,7994.273589,0.0,7.516699e+06,7.332154e+06,184544.396568,29042.45880,29042.4588,0.00000,184544.396568,2.262177e+06,9.447930e+06,2.617476e+06,9.338009e+06,2013-11-01,83559.0,40.06,39,19.0,0.0,0.6000,HL,10901100.0,NaN,NaN,39
1,11220002,12001001.0,1.929025e+07,1.929025e+07,2010-09-15,13.734072,10.612692,137.0,180,1,2010-09-15,MUMBAI,100504.575864,100504.575864,0.0,1.061903e+07,1.033958e+07,279448.084560,0.00000,0.0000,0.00000,279448.084560,1.234760e+07,6.942656e+06,4.945154e+06,1.713861e+07,2013-11-01,126530.0,84.31,38,99.0,0.0,0.6000,HL,10901100.0,NaN,NaN,38
2,11220003,12001002.0,4.053499e+06,4.053499e+06,2010-09-06,17.167590,12.485520,360.0,180,5,2010-09-06,MUMBAI,0.000000,0.000000,0.0,2.439242e+06,2.439242e+06,0.000000,0.44171,0.0000,0.44171,-0.441710,3.944954e+06,1.085444e+05,1.549566e+06,3.944954e+06,2013-11-05,30898.0,79.34,38,322.0,0.0,0.9100,HL,10901101.0,NaN,NaN,38
3,11220004,12001003.0,1.325703e+07,1.325703e+07,2010-09-29,15.606900,10.924830,198.0,180,5,2010-09-29,MUMBAI,28441.732656,28441.732656,0.0,7.211965e+06,7.211965e+06,0.000000,0.00000,0.0000,0.00000,0.000000,1.030278e+07,2.954253e+06,4.219464e+06,1.255472e+07,2013-11-05,88256.0,86.86,37,161.0,0.0,0.9686,HL,10901101.0,NaN,NaN,38
4,11220005,12001004.0,1.321140e+07,1.321140e+07,2010-09-06,16.543314,11.861244,142.0,180,5,2010-09-06,MUMBAI,0.000000,0.000000,0.0,7.712034e+06,7.712034e+06,0.000000,0.00000,0.0000,0.00000,0.000000,8.479158e+06,4.732245e+06,3.812974e+06,1.178201e+07,2013-11-05,91892.0,40.93,38,104.0,0.0,0.7300,HL,10901101.0,NaN,NaN,38


In [3]:

tran=tran1.copy()

### Customer data

In [4]:
cust.head()

,CUSTOMERID,CUST_CONSTTYPE_ID,CUST_CATEGORYID,PROFESSION,AGE,SEX,MARITAL_STATUS,QUALIFICATION,NO_OF_DEPENDENT,OCCUPATION,POSITION,GROSS_INCOME,PRE_JOBYEARS,NETTAKEHOMEINCOME,BRANCH_PINCODE
0,12001000,1,5,NaN,33.0,M,M,POSTGRAD,0.0,NaN,NaN,198375.221829,8.0,198375.221829,400070.0
1,12001001,1,5,NaN,44.0,M,M,POSTGRAD,0.0,NaN,NaN,242703.982915,10.0,242703.982915,400070.0
2,12001002,1,7,NaN,50.0,M,M,GRAD,0.0,NaN,NaN,0.000000,NaN,0.000000,400070.0
3,12001003,3,4,NaN,40.0,M,M,GRAD,0.0,NaN,NaN,365263.511624,NaN,365263.516128,400070.0
4,12001004,3,4,NaN,27.0,M,M,GRAD,0.0,NaN,NaN,187342.198995,NaN,187342.198995,400070.0


### Email Interaction

In [5]:
final.head()

,TicketId,Type,SubType,Status,Date,Preprocessed_EmailBody,Preprocessed_Subject,Masked_CustomerID,Masked_AgreementID
0,571910,REQUEST,PROVISIONAL IT CERTIFICATE,Close,2019-01-23 16:34:00,"hi team, can you please provide me itcertifica...",loan account number - loannumber,12020736,0
1,572747,REQUEST,SOA,Close,2019-01-24 16:05:00,hi team can you please arrange the certificate...,certificate of interest and principal for home...,12019080,0
2,4622,General,Others,Close,2016-02-12 13:40:00,NaN,re: repaysch : loan number loannumber,12004815,0
3,4674,REQUEST,SOA,Close,2016-02-12 15:41:00,"dear sir, you for your promptreply, please not...",re: finalit : loan number loannumber,12005158,11230237
4,4681,QUERY,CHANGEOFROI,Close,2016-02-12 15:55:00,"dear sachin, for your mail. this seems a great...",re: fw: repaysch : loan number loannumber,12001022,0


In [6]:
print(len(np.intersect1d(final['Masked_AgreementID'].values,tran.AGREEMENTID)))
print(len(np.intersect1d(final['Masked_CustomerID'].values,tran.CUSTOMERID)))

401
2709


In [7]:
tran.sample(10)

,AGREEMENTID,CUSTOMERID,LOAN_AMT,NET_DISBURSED_AMT,INTEREST_START_DATE,CURRENT_ROI,ORIGNAL_ROI,CURRENT_TENOR,ORIGNAL_TENOR,DUEDAY,AUTHORIZATIONDATE,CITY,PRE_EMI_DUEAMT,PRE_EMI_RECEIVED_AMT,PRE_EMI_OS_AMOUNT,EMI_DUEAMT,EMI_RECEIVED_AMT,EMI_OS_AMOUNT,EXCESS_AVAILABLE,EXCESS_ADJUSTED_AMT,BALANCE_EXCESS,NET_RECEIVABLE,OUTSTANDING_PRINCIPAL,PAID_PRINCIPAL,PAID_INTEREST,MONTHOPENING,LAST_RECEIPT_DATE,LAST_RECEIPT_AMOUNT,NET_LTV,COMPLETED_TENURE,BALANCE_TENURE,DPD,FOIR,PRODUCT,SCHEMEID,NPA_IN_LAST_MONTH,NPA_IN_CURRENT_MONTH,MOB
186692,11228417,12009416.0,6.294933e+05,6.294933e+05,2016-01-11,11.074656,11.074656,180.0,180,5,2016-01-11,AHMEDABAD,5704.689816,5704.689816,0.0,9.062572e+04,9.062572e+04,0.0,4.432343e+04,4.262505e+04,1698.376488,-1698.376488,5.833030e+05,4.619037e+04,4.826666e+04,6.122780e+05,2016-12-05,4025.0,45.68,10,170.0,0.0,0.39,STHL,10901295.0,NaN,NaN,11
329269,11236054,12017053.0,1.426231e+06,1.426231e+06,2017-08-28,15.856610,15.856610,120.0,120,5,2017-08-28,KURNOOL,5923.336464,5923.336464,0.0,9.287182e+04,9.287182e+04,0.0,0.000000e+00,0.000000e+00,0.000000,0.000000,1.408196e+06,1.803507e+04,4.912240e+04,1.408196e+06,2017-12-05,14017.0,32.88,3,117.0,0.0,0.43,STLAP,10901289.0,NaN,NaN,4
80829,11221939,12002938.0,5.637285e+06,5.637285e+06,2013-05-17,13.734072,13.109796,118.0,180,5,2013-05-17,MUMBAI,45591.138936,45591.138936,0.0,2.728178e+06,2.728178e+06,0.0,1.750991e+06,1.750991e+06,0.000000,0.000000,3.878704e+06,1.758581e+06,1.253914e+06,5.005688e+06,2015-11-05,42823.0,73.73,29,89.0,0.0,0.59,HL,10901104.0,NaN,NaN,30
343726,11235915,12016914.0,2.251944e+06,2.251944e+06,2017-08-29,16.168748,16.168748,229.0,228,5,2017-08-29,NAMAKKAL,49900.023888,49900.023888,0.0,7.827550e+04,7.827550e+04,0.0,0.000000e+00,0.000000e+00,0.000000,0.000000,2.247315e+06,4.628495e+03,8.250071e+04,2.247315e+06,2018-01-05,17721.0,43.60,2,227.0,0.0,0.57,STHL,10901291.0,NaN,NaN,5
283653,11231127,12012126.0,9.007775e+05,9.007775e+05,2016-09-22,15.294762,15.294762,180.0,180,5,2016-09-22,GUNTUR,5863.705560,5863.705560,0.0,1.772981e+05,1.772981e+05,0.0,0.000000e+00,0.000000e+00,0.000000,0.000000,8.803929e+05,2.038459e+04,1.041224e+05,8.803929e+05,2017-09-05,7298.0,12.20,11,169.0,0.0,0.79,LAP,10901113.0,0,0,12
85193,11227027,12008026.0,3.753240e+06,3.753240e+06,2015-07-31,17.479728,17.479728,180.0,180,5,2015-07-31,CHENNAI,10737.979824,10737.979824,0.0,2.205946e+05,2.205946e+05,0.0,0.000000e+00,0.000000e+00,0.000000,0.000000,3.734434e+06,1.880523e+04,1.384465e+05,3.734434e+06,2015-11-05,33294.0,29.57,3,177.0,0.0,0.55,STHL,10901256.0,NaN,NaN,4
108326,11222950,12003949.0,2.317597e+06,2.317597e+06,2013-12-03,15.294762,15.919038,171.0,180,5,2013-12-03,PUNE,2416.155888,2416.155888,0.0,1.149624e+06,1.149624e+06,0.0,4.257867e+04,4.257867e+04,0.000000,0.000000,2.177928e+06,1.396686e+05,6.434479e+05,2.177928e+06,2016-03-05,19279.0,39.31,27,144.0,0.0,0.42,LAP,10901106.0,NaN,NaN,27
364452,11239060,12020059.0,1.899139e+07,1.899139e+07,2017-11-30,11.049685,11.049685,180.0,180,5,2017-11-30,AHMEDABAD,34340.775048,34340.775048,0.0,5.617629e+05,5.617629e+05,0.0,2.606091e+02,2.606091e+02,0.000000,0.000000,1.888927e+07,1.021181e+05,3.030921e+05,1.888927e+07,2018-02-21,NaN,53.38,2,178.0,0.0,0.49,HL,10901381.0,NaN,NaN,3
190545,11232440,12013439.0,1.546335e+06,1.546335e+06,2016-12-31,15.919038,15.919038,156.0,156,5,2016-12-31,NELLORE,0.000000,0.000000,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000,0.000000,1.546335e+06,0.000000e+00,0.000000e+00,1.546335e+06,2016-12-29,12017.0,60.59,0,156.0,0.0,0.40,STHL,10901267.0,NaN,NaN,0
209063,11231858,12012857.0,1.391317e+07,1.391317e+07,2016-10-31,15.606900,15.606900,180.0,180,5,2016-10-31,BANGALORE,35535.601680,35535.601680,0.0,7.568089e+05,7.568089e+05,0.0,0.000000e+00,0.000000e+00,0.000000,0.000000,1.383267e+07,8.049648e+04,4.581114e+05,1.383267e+07,2017-02-05,114224.0,27.73,3,177.0,0.0,2.10,LAP,10901264.0,NaN,NaN,4


In [8]:
def tran_finalagg(val):
    if val in list(set(final['Masked_AgreementID'])):
        return 1
    else:
        return 0
# def tran_finalcust(val):
#     if val in list(set(final['Masked_CustomerID'])):
#         return 1
#     else:
#         return 0
# def tran_cust(val):
#     if val in list(set(cust['CUSTOMERID'])):
#         return 1
#     else:
#         return 0
tran['IS_TRAN_FINAL_AGG']=tran['AGREEMENTID'].apply(tran_finalagg)
# tran['IS_AGG_FINAL_CUST']=tran['AGREEMENTID'].apply(tran_finalcust)
# tran['IS_TRAN_CUST']=tran['AGREEMENTID'].apply(tran_cust)
tran.head()

,AGREEMENTID,CUSTOMERID,LOAN_AMT,NET_DISBURSED_AMT,INTEREST_START_DATE,CURRENT_ROI,ORIGNAL_ROI,CURRENT_TENOR,ORIGNAL_TENOR,DUEDAY,AUTHORIZATIONDATE,CITY,PRE_EMI_DUEAMT,PRE_EMI_RECEIVED_AMT,PRE_EMI_OS_AMOUNT,EMI_DUEAMT,EMI_RECEIVED_AMT,EMI_OS_AMOUNT,EXCESS_AVAILABLE,EXCESS_ADJUSTED_AMT,BALANCE_EXCESS,NET_RECEIVABLE,OUTSTANDING_PRINCIPAL,PAID_PRINCIPAL,PAID_INTEREST,MONTHOPENING,LAST_RECEIPT_DATE,LAST_RECEIPT_AMOUNT,NET_LTV,COMPLETED_TENURE,BALANCE_TENURE,DPD,FOIR,PRODUCT,SCHEMEID,NPA_IN_LAST_MONTH,NPA_IN_CURRENT_MONTH,MOB,IS_TRAN_FINAL_AGG
0,11220001,12001000.0,1.171011e+07,1.171011e+07,2010-08-30,13.734072,10.612692,58.0,120,1,2010-08-29,MUMBAI,7994.273589,7994.273589,0.0,7.516699e+06,7.332154e+06,184544.396568,29042.45880,29042.4588,0.00000,184544.396568,2.262177e+06,9.447930e+06,2.617476e+06,9.338009e+06,2013-11-01,83559.0,40.06,39,19.0,0.0,0.6000,HL,10901100.0,NaN,NaN,39,0
1,11220002,12001001.0,1.929025e+07,1.929025e+07,2010-09-15,13.734072,10.612692,137.0,180,1,2010-09-15,MUMBAI,100504.575864,100504.575864,0.0,1.061903e+07,1.033958e+07,279448.084560,0.00000,0.0000,0.00000,279448.084560,1.234760e+07,6.942656e+06,4.945154e+06,1.713861e+07,2013-11-01,126530.0,84.31,38,99.0,0.0,0.6000,HL,10901100.0,NaN,NaN,38,0
2,11220003,12001002.0,4.053499e+06,4.053499e+06,2010-09-06,17.167590,12.485520,360.0,180,5,2010-09-06,MUMBAI,0.000000,0.000000,0.0,2.439242e+06,2.439242e+06,0.000000,0.44171,0.0000,0.44171,-0.441710,3.944954e+06,1.085444e+05,1.549566e+06,3.944954e+06,2013-11-05,30898.0,79.34,38,322.0,0.0,0.9100,HL,10901101.0,NaN,NaN,38,0
3,11220004,12001003.0,1.325703e+07,1.325703e+07,2010-09-29,15.606900,10.924830,198.0,180,5,2010-09-29,MUMBAI,28441.732656,28441.732656,0.0,7.211965e+06,7.211965e+06,0.000000,0.00000,0.0000,0.00000,0.000000,1.030278e+07,2.954253e+06,4.219464e+06,1.255472e+07,2013-11-05,88256.0,86.86,37,161.0,0.0,0.9686,HL,10901101.0,NaN,NaN,38,0
4,11220005,12001004.0,1.321140e+07,1.321140e+07,2010-09-06,16.543314,11.861244,142.0,180,5,2010-09-06,MUMBAI,0.000000,0.000000,0.0,7.712034e+06,7.712034e+06,0.000000,0.00000,0.0000,0.00000,0.000000,8.479158e+06,4.732245e+06,3.812974e+06,1.178201e+07,2013-11-05,91892.0,40.93,38,104.0,0.0,0.7300,HL,10901101.0,NaN,NaN,38,0


In [9]:
tran.columns

Index(['AGREEMENTID', 'CUSTOMERID', 'LOAN_AMT', 'NET_DISBURSED_AMT',
       'INTEREST_START_DATE', 'CURRENT_ROI', 'ORIGNAL_ROI', 'CURRENT_TENOR',
       'ORIGNAL_TENOR', 'DUEDAY', 'AUTHORIZATIONDATE', 'CITY',
       'PRE_EMI_DUEAMT', 'PRE_EMI_RECEIVED_AMT', 'PRE_EMI_OS_AMOUNT',
       'EMI_DUEAMT', 'EMI_RECEIVED_AMT', 'EMI_OS_AMOUNT', 'EXCESS_AVAILABLE',
       'EXCESS_ADJUSTED_AMT', 'BALANCE_EXCESS', 'NET_RECEIVABLE',
       'OUTSTANDING_PRINCIPAL', 'PAID_PRINCIPAL', 'PAID_INTEREST',
       'MONTHOPENING', 'LAST_RECEIPT_DATE', 'LAST_RECEIPT_AMOUNT', 'NET_LTV',
       'COMPLETED_TENURE', 'BALANCE_TENURE', 'DPD', 'FOIR', 'PRODUCT',
       'SCHEMEID', 'NPA_IN_LAST_MONTH', 'NPA_IN_CURRENT_MONTH', 'MOB',
       'IS_TRAN_FINAL_AGG'],
      dtype='object')

In [10]:
# pd.get_dummies(tran,columns=['PRODUCT'],drop_first=True).head().value
tran['CITY'].value_counts()

MUMBAI              71980
HYDERABAD           56724
PUNE                44538
AHMEDABAD           43812
SURAT               41441
CHENNAI             35428
BANGALORE           28041
DELHI               20421
THANE               19831
COIMBATORE          17552
MADURAI             16091
RAJKOT              12191
ERODE                8890
TIRUCHIRAPPALLI      8392
NASHIK               8249
TIRUPUR              7662
WARANGAL             7493
VISHAKHAPATNAM       7195
VADODARA             6803
NAGPUR               6802
KARUR                6650
NAMAKKAL             6578
TUTICORIN            6313
NAVI MUMBAI          6218
NELLORE              6030
PONDICHERRY          5909
SALEM                5534
TIRUNELVELI          4556
MYSORE               4429
VELLORE              4342
                    ...  
ERNAKULAM               7
BULDHANA                7
VETTAIKARANPUDUR        7
AMBAJI                  7
PALWAL                  7
MAHAD                   6
CHAMARAJANAGAR          6
BHANDARA    

### Feature Engineering

In [11]:
from sklearn.preprocessing import LabelEncoder
l1=LabelEncoder()
tran['CITY'].fillna('OTHERS',inplace=True)
tran['CITY']=l1.fit_transform(tran['CITY'])
# tran['CUSTOMERID']=tran['CUSTOMERID'].astype(np.object)
# tran['SCHEMEID']=tran['SCHEMEID'].astype(np.object)
tran=pd.get_dummies(tran,columns=['PRODUCT'],drop_first=True)
tran['last_auth_d']=((tran['LAST_RECEIPT_DATE']-tran['AUTHORIZATIONDATE'])/np.timedelta64(1,'D')).fillna(0)
tran['last_auth_m']=((tran['LAST_RECEIPT_DATE']-tran['AUTHORIZATIONDATE'])/np.timedelta64(1,'M')).fillna(0)
tran['last_auth_y']=((tran['LAST_RECEIPT_DATE']-tran['AUTHORIZATIONDATE'])/np.timedelta64(1,'Y')).fillna(0)

tran['last_int_d']=((tran['LAST_RECEIPT_DATE']-tran['INTEREST_START_DATE'])/np.timedelta64(1,'D')).fillna(0)
tran['last_int_m']=((tran['LAST_RECEIPT_DATE']-tran['INTEREST_START_DATE'])/np.timedelta64(1,'M')).fillna(0)
tran['last_int_y']=((tran['LAST_RECEIPT_DATE']-tran['INTEREST_START_DATE'])/np.timedelta64(1,'Y')).fillna(0)

tran['auth_int_d']=((tran['AUTHORIZATIONDATE']-tran['INTEREST_START_DATE'])/np.timedelta64(1,'D')).fillna(0)
tran['auth_int_m']=((tran['AUTHORIZATIONDATE']-tran['INTEREST_START_DATE'])/np.timedelta64(1,'M')).fillna(0)
tran['auth_int_y']=((tran['AUTHORIZATIONDATE']-tran['INTEREST_START_DATE'])/np.timedelta64(1,'Y')).fillna(0)

# tran['CUSTOMERID']=tran['CUSTOMERID'].astype(np.object)
tran['CUSTOMERID'].fillna(12009999,inplace=True)
# tran['SCHEMEID']=tran['SCHEMEID'].astype(np.object)
tran['SCHEMEID'].fillna(10909999,inplace=True)
tran['LAST_RECEIPT_AMOUNT'].fillna(tran['LAST_RECEIPT_AMOUNT'].mean(),inplace=True)

tran['ROI_DIFF']=tran['CURRENT_ROI']-tran['ORIGNAL_ROI']
tran['TENOR_DIFF']=tran['CURRENT_TENOR']-tran['ORIGNAL_TENOR']
tran['AMT_DIFF']=tran['LOAN_AMT']-tran['NET_DISBURSED_AMT']
# tran.isnull().sum()

In [12]:
tran.isnull().sum()

AGREEMENTID                   0
CUSTOMERID                    0
LOAN_AMT                      0
NET_DISBURSED_AMT             0
INTEREST_START_DATE           0
CURRENT_ROI                   0
ORIGNAL_ROI                   0
CURRENT_TENOR                 5
ORIGNAL_TENOR                 0
DUEDAY                        0
AUTHORIZATIONDATE             0
CITY                          0
PRE_EMI_DUEAMT                0
PRE_EMI_RECEIVED_AMT          0
PRE_EMI_OS_AMOUNT             0
EMI_DUEAMT                    0
EMI_RECEIVED_AMT              0
EMI_OS_AMOUNT                 0
EXCESS_AVAILABLE              0
EXCESS_ADJUSTED_AMT           0
BALANCE_EXCESS                0
NET_RECEIVABLE                0
OUTSTANDING_PRINCIPAL         0
PAID_PRINCIPAL                0
PAID_INTEREST                 0
MONTHOPENING                  0
LAST_RECEIPT_DATE          3787
LAST_RECEIPT_AMOUNT           0
NET_LTV                       0
COMPLETED_TENURE              0
BALANCE_TENURE                5
DPD     

In [13]:
tran.describe(include='all').T


,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max
AGREEMENTID,624250,NaN,NaN,NaN,NaN,NaN,1.12308e+07,7984.21,1.122e+07,1.12243e+07,1.12289e+07,1.12359e+07,1.12534e+07
CUSTOMERID,624250,NaN,NaN,NaN,NaN,NaN,1.20118e+07,7958.61,1.2001e+07,1.20053e+07,1.20099e+07,1.20169e+07,1.20339e+07
LOAN_AMT,624250,NaN,NaN,NaN,NaN,NaN,5.8282e+06,1.32312e+07,37532.4,1.51631e+06,2.62727e+06,5.20973e+06,5.70492e+08
NET_DISBURSED_AMT,624250,NaN,NaN,NaN,NaN,NaN,5.77819e+06,1.31355e+07,37532.4,1.5013e+06,2.59321e+06,5.1669e+06,5.67459e+08
INTEREST_START_DATE,624250,1847,2018-03-31 00:00:00,8081,2010-08-30 00:00:00,2018-12-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CURRENT_ROI,624250,NaN,NaN,NaN,NaN,NaN,15.0688,2.50616,-5.05664,13.1098,14.9826,17.1676,37.4566
ORIGNAL_ROI,624250,NaN,NaN,NaN,NaN,NaN,14.9971,2.47094,0,13.1098,14.9826,16.5433,34.1479
CURRENT_TENOR,624245,NaN,NaN,NaN,NaN,NaN,178.406,49.4907,0,150,180,203,713
ORIGNAL_TENOR,624250,NaN,NaN,NaN,NaN,NaN,177.91,44.2123,0,168,180,180,300
DUEDAY,624250,NaN,NaN,NaN,NaN,NaN,6.40512,3.53397,1,5,5,5,15


**Building more features**

In [14]:
['mean','sum','min','max','median','quantile','mad','std','count','nunique']
ag_cat=['sum', 'mean','count','nunique','min','max']
ag_num=['mean','sum','min','max','median','quantile','mad','std']
agg_func = {
    'CUSTOMERID': ag_cat,
    'LOAN_AMT': ['mean','sum','min','max','median','quantile','mad','std'],
    'NET_DISBURSED_AMT': ag_num,
#     'INTEREST_START_DATE': ag_num,
    'CURRENT_ROI': ag_num,
    'ORIGNAL_ROI': ag_num,
    'CURRENT_TENOR': ag_num,
    'ORIGNAL_TENOR': ag_num,
    'DUEDAY': ag_cat,
#     'AUTHORIZATIONDATE': ag_num,
    'CITY': ag_cat,
    'PRE_EMI_DUEAMT': ag_num,
    'PRE_EMI_RECEIVED_AMT': ag_num,
    'PRE_EMI_OS_AMOUNT': ag_num,
    'EMI_DUEAMT': ag_num,
    'EMI_RECEIVED_AMT': ag_num,
    'EMI_OS_AMOUNT': ag_num,
    'EXCESS_AVAILABLE': ag_num,
    'EXCESS_ADJUSTED_AMT': ag_num,
    'BALANCE_EXCESS': ag_num,
    'NET_RECEIVABLE': ag_num,
    'OUTSTANDING_PRINCIPAL': ag_num,
    'PAID_PRINCIPAL': ag_num,
    'PAID_INTEREST': ag_num,
    'MONTHOPENING': ag_num,
#     'LAST_RECEIPT_DATE': ag_num,
    'LAST_RECEIPT_AMOUNT':ag_num,
    'NET_LTV': ag_num,
    'COMPLETED_TENURE': ag_num,
    'BALANCE_TENURE': ag_num,
    'DPD': ag_num,
    'last_auth_y': ag_num,
    'last_int_d': ag_num,
    'last_int_m':ag_num,
    'last_int_y': ag_num,
    'auth_int_d': ag_num,
    'auth_int_m':ag_num,
    'auth_int_y': ag_num,
    'FOIR': ag_num,
    'SCHEMEID': ag_cat,
    'ROI_DIFF': ag_num,
    'TENOR_DIFF': ag_num,
    'AMT_DIFF':ag_num,
    'IS_TRAN_FINAL_AGG': ag_num,
    'MOB':ag_cat,
    'PRODUCT_STHL': ag_num,
    'PRODUCT_STLAP': ag_num,
    'PRODUCT_LAP':ag_num,
    'last_auth_d':ag_num,
    'last_auth_m':ag_num,
    'NPA_IN_LAST_MONTH':['count','nunique'],
    'NPA_IN_CURRENT_MONTH':['count','nunique']
    }

print(len(agg_func))
print(len(tran.columns))

49
53


Combining dataset

In [15]:
df=tran.groupby(by=['AGREEMENTID']).agg(agg_func)
df.columns=['LMS' + '_'.join(col).strip() for col in df.columns.values]
df.reset_index(inplace=True)
# df=df.append(tran.groupby(by=['AGREEMENTID']).std().reset_index())
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33354 entries, 0 to 33353
Columns: 371 entries, AGREEMENTID to LMSNPA_IN_CURRENT_MONTH_nunique
dtypes: float64(320), int64(37), uint8(14)
memory usage: 91.3 MB


In [18]:
df.head()


,AGREEMENTID,LMSCUSTOMERID_sum,LMSCUSTOMERID_mean,LMSCUSTOMERID_count,LMSCUSTOMERID_nunique,LMSCUSTOMERID_min,LMSCUSTOMERID_max,LMSLOAN_AMT_mean,LMSLOAN_AMT_sum,LMSLOAN_AMT_min,LMSLOAN_AMT_max,LMSLOAN_AMT_median,LMSLOAN_AMT_quantile,LMSLOAN_AMT_mad,LMSLOAN_AMT_std,LMSNET_DISBURSED_AMT_mean,LMSNET_DISBURSED_AMT_sum,LMSNET_DISBURSED_AMT_min,LMSNET_DISBURSED_AMT_max,LMSNET_DISBURSED_AMT_median,LMSNET_DISBURSED_AMT_quantile,LMSNET_DISBURSED_AMT_mad,LMSNET_DISBURSED_AMT_std,LMSCURRENT_ROI_mean,LMSCURRENT_ROI_sum,LMSCURRENT_ROI_min,LMSCURRENT_ROI_max,LMSCURRENT_ROI_median,LMSCURRENT_ROI_quantile,LMSCURRENT_ROI_mad,LMSCURRENT_ROI_std,LMSORIGNAL_ROI_mean,LMSORIGNAL_ROI_sum,LMSORIGNAL_ROI_min,LMSORIGNAL_ROI_max,LMSORIGNAL_ROI_median,LMSORIGNAL_ROI_quantile,LMSORIGNAL_ROI_mad,LMSORIGNAL_ROI_std,LMSCURRENT_TENOR_mean,...,LMSPRODUCT_STHL_median,LMSPRODUCT_STHL_quantile,LMSPRODUCT_STHL_mad,LMSPRODUCT_STHL_std,LMSPRODUCT_STLAP_mean,LMSPRODUCT_STLAP_sum,LMSPRODUCT_STLAP_min,LMSPRODUCT_STLAP_max,LMSPRODUCT_STLAP_median,LMSPRODUCT_STLAP_quantile,LMSPRODUCT_STLAP_mad,LMSPRODUCT_STLAP_std,LMSPRODUCT_LAP_mean,LMSPRODUCT_LAP_sum,LMSPRODUCT_LAP_min,LMSPRODUCT_LAP_max,LMSPRODUCT_LAP_median,LMSPRODUCT_LAP_quantile,LMSPRODUCT_LAP_mad,LMSPRODUCT_LAP_std,LMSlast_auth_d_mean,LMSlast_auth_d_sum,LMSlast_auth_d_min,LMSlast_auth_d_max,LMSlast_auth_d_median,LMSlast_auth_d_quantile,LMSlast_auth_d_mad,LMSlast_auth_d_std,LMSlast_auth_m_mean,LMSlast_auth_m_sum,LMSlast_auth_m_min,LMSlast_auth_m_max,LMSlast_auth_m_median,LMSlast_auth_m_quantile,LMSlast_auth_m_mad,LMSlast_auth_m_std,LMSNPA_IN_LAST_MONTH_count,LMSNPA_IN_LAST_MONTH_nunique,LMSNPA_IN_CURRENT_MONTH_count,LMSNPA_IN_CURRENT_MONTH_nunique
0,11220001,84007000.0,12001000.0,7,1,12001000.0,12001000.0,1.171011e+07,8.197075e+07,1.171011e+07,1.171011e+07,1.171011e+07,1.171011e+07,1.862645e-09,0.0,1.171011e+07,8.197075e+07,1.171011e+07,1.171011e+07,1.171011e+07,1.171011e+07,1.862645e-09,0.0,13.511116,94.577814,13.421934,13.734072,13.421934,13.421934,1.274033e-01,0.152308,10.612692,74.288844,10.612692,10.612692,10.612692,10.612692,0.000000e+00,0.0,56.142857,...,0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,1251.285714,8759.0,1160.0,1345.0,1252.0,1252.0,52.244898,66.054811,41.110847,287.775930,38.111666,44.189819,41.134315,41.134315,1.716500,2.170223,0,0,0,0
1,11220002,12001001.0,12001001.0,1,1,12001001.0,12001001.0,1.929025e+07,1.929025e+07,1.929025e+07,1.929025e+07,1.929025e+07,1.929025e+07,0.000000e+00,NaN,1.929025e+07,1.929025e+07,1.929025e+07,1.929025e+07,1.929025e+07,1.929025e+07,0.000000e+00,NaN,13.734072,13.734072,13.734072,13.734072,13.734072,13.734072,0.000000e+00,NaN,10.612692,10.612692,10.612692,10.612692,10.612692,10.612692,0.000000e+00,NaN,137.000000,...,0,0.0,0.0,NaN,0,0,0,0,0,0.0,0.0,NaN,0.0,0,0,0,0,0.0,0.0,NaN,1143.000000,1143.0,1143.0,1143.0,1143.0,1143.0,0.000000,NaN,37.553133,37.553133,37.553133,37.553133,37.553133,37.553133,0.000000,NaN,0,0,0,0
2,11220003,600050100.0,12001002.0,50,1,12001002.0,12001002.0,4.053499e+06,2.026749e+08,4.053499e+06,4.053499e+06,4.053499e+06,4.053499e+06,3.259629e-09,0.0,4.053499e+06,2.026749e+08,4.053499e+06,4.053499e+06,4.053499e+06,4.053499e+06,3.259629e-09,0.0,16.949093,847.454670,16.543314,17.167590,16.855452,16.855452,2.097567e-01,0.238052,12.485520,624.276000,12.485520,12.485520,12.485520,12.485520,3.552714e-15,0.0,340.640000,...,0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,1902.340000,95117.0,1156.0,2647.0,1901.0,1901.0,381.780000,444.576185,62.501160,3125.058009,37.980246,86.966878,62.457135,62.457135,12.543338,14.606499,1,1,1,1
3,11220004,12001003.0,12001003.0,1,1,12001003.0,12001003.0,1.325703e+07,1.325703e+07,1.325703e+07,1.325703e+07,1.325703e+07,1.325703e+07,0.000000e+00,NaN,1.325703e+07,1.325703e+07,1.325703e+07,1.325703e+07,1.325703e+07,1.325703e+07,0.000000e+00,NaN,15.606900,15.606900,15.606900,15.606900,15.606900,15.606900,0.000000e+00,NaN,10.924830,10.924830,10.924830,10.924830,10.924830,10.924830,0.000000e+00,NaN,198.000

In [19]:
# cust1=cust.drop(['PROFESSION','OCCUPATION','POSITION','PRE_JOBYEARS'],axis=1)
# cust1.head()
df.isnull().sum()

AGREEMENTID                           0
LMSCUSTOMERID_sum                     0
LMSCUSTOMERID_mean                    0
LMSCUSTOMERID_count                   0
LMSCUSTOMERID_nunique                 0
LMSCUSTOMERID_min                     0
LMSCUSTOMERID_max                     0
LMSLOAN_AMT_mean                      0
LMSLOAN_AMT_sum                       0
LMSLOAN_AMT_min                       0
LMSLOAN_AMT_max                       0
LMSLOAN_AMT_median                    0
LMSLOAN_AMT_quantile                  0
LMSLOAN_AMT_mad                       0
LMSLOAN_AMT_std                    1086
LMSNET_DISBURSED_AMT_mean             0
LMSNET_DISBURSED_AMT_sum              0
LMSNET_DISBURSED_AMT_min              0
LMSNET_DISBURSED_AMT_max              0
LMSNET_DISBURSED_AMT_median           0
LMSNET_DISBURSED_AMT_quantile         0
LMSNET_DISBURSED_AMT_mad              0
LMSNET_DISBURSED_AMT_std           1086
LMSCURRENT_ROI_mean                   0
LMSCURRENT_ROI_sum                    0


In [20]:
# print(len(np.intersect1d(cust1['CUSTOMERID'].values,df_train.CUSTOMERID)))

In [21]:
# # cust1['BRANCH_PINCODE'].value_counts()
# # cust1
# cust1['AGE'].fillna(cust1['AGE'].mean(),inplace=True)
# cust1['SEX'].fillna('O',inplace=True)
# cust1['MARITAL_STATUS'].fillna('O',inplace=True)
# cust1['NO_OF_DEPENDENT'].fillna(0,inplace=True)
# cust1['QUALIFICATION'].fillna('OTHER',inplace=True)
# cust1['BRANCH_PINCODE']=cust1['BRANCH_PINCODE'].astype(np.object)
# cust1['CUST_CONSTTYPE_ID']=cust1['CUST_CONSTTYPE_ID'].astype(np.object)
# cust1['CUST_CATEGORYID']=cust1['CUST_CATEGORYID'].astype(np.object)
# cust1['BRANCH_PINCODE'].fillna('0',inplace=True)
# cust1.info()

In [22]:
df_train=df.merge(train,on=['AGREEMENTID'])
df_test=df.merge(test,on=['AGREEMENTID']).drop('FORECLOSURE',axis=1)
# del df_test

In [23]:
df_train.head()

,AGREEMENTID,LMSCUSTOMERID_sum,LMSCUSTOMERID_mean,LMSCUSTOMERID_count,LMSCUSTOMERID_nunique,LMSCUSTOMERID_min,LMSCUSTOMERID_max,LMSLOAN_AMT_mean,LMSLOAN_AMT_sum,LMSLOAN_AMT_min,LMSLOAN_AMT_max,LMSLOAN_AMT_median,LMSLOAN_AMT_quantile,LMSLOAN_AMT_mad,LMSLOAN_AMT_std,LMSNET_DISBURSED_AMT_mean,LMSNET_DISBURSED_AMT_sum,LMSNET_DISBURSED_AMT_min,LMSNET_DISBURSED_AMT_max,LMSNET_DISBURSED_AMT_median,LMSNET_DISBURSED_AMT_quantile,LMSNET_DISBURSED_AMT_mad,LMSNET_DISBURSED_AMT_std,LMSCURRENT_ROI_mean,LMSCURRENT_ROI_sum,LMSCURRENT_ROI_min,LMSCURRENT_ROI_max,LMSCURRENT_ROI_median,LMSCURRENT_ROI_quantile,LMSCURRENT_ROI_mad,LMSCURRENT_ROI_std,LMSORIGNAL_ROI_mean,LMSORIGNAL_ROI_sum,LMSORIGNAL_ROI_min,LMSORIGNAL_ROI_max,LMSORIGNAL_ROI_median,LMSORIGNAL_ROI_quantile,LMSORIGNAL_ROI_mad,LMSORIGNAL_ROI_std,LMSCURRENT_TENOR_mean,...,LMSPRODUCT_STHL_quantile,LMSPRODUCT_STHL_mad,LMSPRODUCT_STHL_std,LMSPRODUCT_STLAP_mean,LMSPRODUCT_STLAP_sum,LMSPRODUCT_STLAP_min,LMSPRODUCT_STLAP_max,LMSPRODUCT_STLAP_median,LMSPRODUCT_STLAP_quantile,LMSPRODUCT_STLAP_mad,LMSPRODUCT_STLAP_std,LMSPRODUCT_LAP_mean,LMSPRODUCT_LAP_sum,LMSPRODUCT_LAP_min,LMSPRODUCT_LAP_max,LMSPRODUCT_LAP_median,LMSPRODUCT_LAP_quantile,LMSPRODUCT_LAP_mad,LMSPRODUCT_LAP_std,LMSlast_auth_d_mean,LMSlast_auth_d_sum,LMSlast_auth_d_min,LMSlast_auth_d_max,LMSlast_auth_d_median,LMSlast_auth_d_quantile,LMSlast_auth_d_mad,LMSlast_auth_d_std,LMSlast_auth_m_mean,LMSlast_auth_m_sum,LMSlast_auth_m_min,LMSlast_auth_m_max,LMSlast_auth_m_median,LMSlast_auth_m_quantile,LMSlast_auth_m_mad,LMSlast_auth_m_std,LMSNPA_IN_LAST_MONTH_count,LMSNPA_IN_LAST_MONTH_nunique,LMSNPA_IN_CURRENT_MONTH_count,LMSNPA_IN_CURRENT_MONTH_nunique,FORECLOSURE
0,11220001,84007000.0,12001000.0,7,1,12001000.0,12001000.0,1.171011e+07,8.197075e+07,1.171011e+07,1.171011e+07,1.171011e+07,1.171011e+07,1.862645e-09,0.0,1.171011e+07,8.197075e+07,1.171011e+07,1.171011e+07,1.171011e+07,1.171011e+07,1.862645e-09,0.0,13.511116,94.577814,13.421934,13.734072,13.421934,13.421934,0.127403,0.152308,10.612692,74.288844,10.612692,10.612692,10.612692,10.612692,0.000000e+00,0.0,56.142857,...,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,1251.285714,8759.0,1160.0,1345.0,1252.0,1252.0,52.244898,66.054811,41.110847,287.775930,38.111666,44.189819,41.134315,41.134315,1.716500,2.170223,0,0,0,0,1
1,11220002,12001001.0,12001001.0,1,1,12001001.0,12001001.0,1.929025e+07,1.929025e+07,1.929025e+07,1.929025e+07,1.929025e+07,1.929025e+07,0.000000e+00,NaN,1.929025e+07,1.929025e+07,1.929025e+07,1.929025e+07,1.929025e+07,1.929025e+07,0.000000e+00,NaN,13.734072,13.734072,13.734072,13.734072,13.734072,13.734072,0.000000,NaN,10.612692,10.612692,10.612692,10.612692,10.612692,10.612692,0.000000e+00,NaN,137.000000,...,0.0,0.0,NaN,0,0,0,0,0,0.0,0.0,NaN,0.0,0,0,0,0,0.0,0.0,NaN,1143.000000,1143.0,1143.0,1143.0,1143.0,1143.0,0.000000,NaN,37.553133,37.553133,37.553133,37.553133,37.553133,37.553133,0.000000,NaN,0,0,0,0,1
2,11220006,552046230.0,12001005.0,46,1,12001005.0,12001005.0,3.933395e+06,1.809362e+08,3.933395e+06,3.933395e+06,3.933395e+06,3.933395e+06,0.000000e+00,0.0,3.933395e+06,1.809362e+08,3.933395e+06,3.933395e+06,3.933395e+06,3.933395e+06,0.000000e+00,0.0,16.047965,738.206370,15.606900,16.231176,16.231176,16.231176,0.191177,0.214002,11.549106,531.258876,11.549106,11.549106,11.549106,11.549106,5.329071e-15,0.0,339.108696,...,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0.0,0.0,1783.608696,82046.0,1099.0,2468.0,1783.0,1783.0,349.739130,408.524417,58.600257,2695.611820,36.107518,81.085854,58.580258,58.580258,11.490639,13.422022,0,0,0,0,1
3,11220008,660055385.0,12001007.0,55,1,12001007.0,12001007.0,1.002259e+07,5.512423e+08,1.002259e+07,1.002259e+07,1.002259e+07,1.002259e+07,9.313226e-09,0.0,1.002259e+07,5.512423e+08,1.002259e+07,1.002259e+07,1.002259e+07,1.002259e+07,9.313226e-09,0.0,12.797658,703.871190,11.236968,13.734072,12.797658,12.797658,0.567524,0.778609,10.612692,583.698060,10.612692,10.612692,10.612692,10.612692,1.776357e-15,0.0,113.709091,...,0.0,0.0,0.

In [24]:
df_train.isnull().sum()

AGREEMENTID                          0
LMSCUSTOMERID_sum                    0
LMSCUSTOMERID_mean                   0
LMSCUSTOMERID_count                  0
LMSCUSTOMERID_nunique                0
LMSCUSTOMERID_min                    0
LMSCUSTOMERID_max                    0
LMSLOAN_AMT_mean                     0
LMSLOAN_AMT_sum                      0
LMSLOAN_AMT_min                      0
LMSLOAN_AMT_max                      0
LMSLOAN_AMT_median                   0
LMSLOAN_AMT_quantile                 0
LMSLOAN_AMT_mad                      0
LMSLOAN_AMT_std                    665
LMSNET_DISBURSED_AMT_mean            0
LMSNET_DISBURSED_AMT_sum             0
LMSNET_DISBURSED_AMT_min             0
LMSNET_DISBURSED_AMT_max             0
LMSNET_DISBURSED_AMT_median          0
LMSNET_DISBURSED_AMT_quantile        0
LMSNET_DISBURSED_AMT_mad             0
LMSNET_DISBURSED_AMT_std           665
LMSCURRENT_ROI_mean                  0
LMSCURRENT_ROI_sum                   0
LMSCURRENT_ROI_min       

In [25]:
# tran['PRODUCT'].value_counts()
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13342 entries, 0 to 13341
Columns: 371 entries, AGREEMENTID to LMSNPA_IN_CURRENT_MONTH_nunique
dtypes: float64(320), int64(37), uint8(14)
memory usage: 36.6 MB


In [26]:
# df_train['LOAN_AMT']-df_train['NET_DISBURSED_AMT']
# df_train.isnull().sum()

In [27]:
# df_train['CUSTOMERID']=df_train['CUSTOMERID'].astype(np.object)
# df_train['CUSTOMERID'].fillna('1200XXXX',inplace=True)
# df_train['SCHEMEID']=df_train['SCHEMEID'].astype(np.object)
# df_train['SCHEMEID'].fillna('1090XXXX',inplace=True)
# df_train['LAST_RECEIPT_AMOUNT'].fillna(df_train['LAST_RECEIPT_AMOUNT'].mean(),inplace=True)

# df_train['ROI_DIFF']=df_train['CURRENT_ROI']-df_train['ORIGNAL_ROI']
# df_train['TENOR_DIFF']=df_train['CURRENT_TENOR']-df_train['ORIGNAL_TENOR']
# df_train['AMT_DIFF']=df_train['LOAN_AMT']-df_train['NET_DISBURSED_AMT']

# df_test['CUSTOMERID']=df_test['CUSTOMERID'].astype(np.object)
# df_test['CUSTOMERID'].fillna('1200XXXX',inplace=True)
# df_test['SCHEMEID']=df_test['SCHEMEID'].astype(np.object)
# df_test['SCHEMEID'].fillna('1090XXXX',inplace=True)
# df_test['LAST_RECEIPT_AMOUNT'].fillna(df_test['LAST_RECEIPT_AMOUNT'].mean(),inplace=True)
# df_test['DPD'].fillna(df_test['DPD'].mean(),inplace=True)
# df_test['ROI_DIFF']=df_test['CURRENT_ROI']-df_test['ORIGNAL_ROI']
# df_test['TENOR_DIFF']=df_test['CURRENT_TENOR']-df_test['ORIGNAL_TENOR']
# df_test['AMT_DIFF']=df_test['LOAN_AMT']-df_test['NET_DISBURSED_AMT']

### Evaluating and Predicting

In [28]:
from catboost import CatBoostClassifier,Pool, cv
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold,train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,roc_auc_score

In [29]:
X_train,X_val,y_train,y_val = train_test_split(df_train.drop(['AGREEMENTID','FORECLOSURE'],axis=1),df_train['FORECLOSURE'],
                                                 test_size=0.25,random_state = 1994)

In [30]:
# y.value_counts()

In [31]:
categorical_features_indices = np.where(X_train.dtypes =='object')[0]
categorical_features_indices

array([], dtype=int64)

In [32]:
# cat_model = CatBoostClassifier(n_estimators=1000, # use large n_estimators deliberately to make use of the early stopping
# #                          reg_lambda=0.25,
# #                          l2_leaf_reg=10.0,
# #                         custom_metric= ['AUC','Logloss'],
#                          eval_metric='AUC',
#                          random_seed=1994,
# #                          learning_rate = 0.12,
#                          depth = 6,
# #                                class_weights=[1,20],
# #                                boosting_type = 'Ordered',
# #                          subsample = 0.8
#                          #rsm = 0.7,
#                          #silent=True,
#                          #max_ctr_complexity = 5,  # no of categorical cols combined
# #                          boosting_type = 'Ordered',
# #                          od_type = 'IncToDec',  #overfitting params
# #                          od_wait = 20,
#                          bagging_temperature = 1.0
#                               )
# # lr=0.05, no od type of vars -- highest
    
# cat_model.fit(X_train.values,y_train.values,cat_features=categorical_features_indices,eval_set=(X_val, y_val),
#         plot=False,early_stopping_rounds=100,use_best_model=True,verbose=50) # early stopping set to 100 to prevent overfitting

In [33]:
# m=LGBMClassifier(n_estimators=1500,random_state=1994,learning_rate=0.03,max_depth=7,num_leaves=100,reg_alpha=2)
m=LGBMClassifier(n_estimators=1500,random_state=1994)
# m=RidgeCV(cv=4)
m.fit(X_train,y_train,eval_set=[(X_val, y_val.values)],eval_metric='auc', early_stopping_rounds=100,verbose=100)
p=m.predict_proba(X_val)[:,-1]
# m=Rid
# print(np.sqrt(mean_squared_log_error(np.expm1(y_val.values),np.expm1(p))))
# print(np.sqrt(mean_squared_log_error(y_val.values,p)))
print(roc_auc_score(y_val,p))

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.995478	valid_0's binary_logloss: 0.0280304
Early stopping, best iteration is:
[77]	valid_0's auc: 0.995347	valid_0's binary_logloss: 0.0277418
0.9953467724328652


In [34]:
sorted(zip(m.feature_importances_,X_train),reverse=True)

[(371, 'LMSCUSTOMERID_mean'),
 (191, 'LMSSCHEMEID_mean'),
 (159, 'LMSCUSTOMERID_sum'),
 (85, 'LMSPRODUCT_LAP_mean'),
 (58, 'LMSPRODUCT_STLAP_mean'),
 (48, 'LMSPRODUCT_STHL_sum'),
 (34, 'LMSNET_LTV_mean'),
 (32, 'LMSROI_DIFF_std'),
 (32, 'LMSNET_LTV_sum'),
 (30, 'LMSROI_DIFF_sum'),
 (30, 'LMSFOIR_mean'),
 (28, 'LMSPAID_INTEREST_sum'),
 (24, 'LMSPRODUCT_LAP_sum'),
 (23, 'LMSlast_int_d_min'),
 (23, 'LMSlast_auth_y_sum'),
 (23, 'LMSOUTSTANDING_PRINCIPAL_mad'),
 (22, 'LMSlast_auth_y_median'),
 (21, 'LMSCUSTOMERID_count'),
 (21, 'LMSCURRENT_ROI_mean'),
 (20, 'LMSlast_int_d_sum'),
 (20, 'LMSlast_auth_y_mean'),
 (20, 'LMSauth_int_d_mean'),
 (20, 'LMSBALANCE_TENURE_min'),
 (19, 'LMSauth_int_d_mad'),
 (19, 'LMSLAST_RECEIPT_AMOUNT_std'),
 (19, 'LMSCURRENT_ROI_mad'),
 (18, 'LMSlast_auth_y_min'),
 (18, 'LMSLAST_RECEIPT_AMOUNT_mean'),
 (18, 'LMSBALANCE_TENURE_mean'),
 (17, 'LMSSCHEMEID_sum'),
 (17, 'LMSCITY_sum'),
 (16, 'LMSPRODUCT_STLAP_sum'),
 (15, 'LMSlast_auth_y_std'),
 (15, 'LMSLAST_RECEIPT_AMO

In [35]:
# preds = cat_model.predict_proba(X_val)[:,1]
# print(roc_auc_score(y_val,preds))

In [36]:
#LGB
# X_train,X_val,y_train,y_val = train_test_split(df_train.drop(['AGREEMENTID','FORECLOSURE','CUSTOMERID','SCHEMEID'],axis=1),df_train['FORECLOSURE'],
#                                                  test_size=0.25,random_state = 1994)
# import lightgbm as lgb
# from lightgbm import LGBMModel,LGBMClassifier

# X,y=df_train.drop(['AGREEMENTID','FORECLOSURE','CUSTOMERID','SCHEMEID'],axis=1),df_train['FORECLOSURE']
# Xtest=df_test.drop(['AGREEMENTID','CUSTOMERID','SCHEMEID'],axis=1)

# lgbc = LGBMClassifier(n_estimators=300, random_state =1994,metrics ='auc')
# fit_model = lgbc.fit(X, y)
# y_test_pred = fit_model.predict_proba(Xtest)[:,1]
# print(roc_auc_score(y_val,y_test_pred))

In [37]:
X,y=df_train.drop(['AGREEMENTID','FORECLOSURE'],axis=1),df_train['FORECLOSURE']
Xtest=df_test.drop(['AGREEMENTID'],axis=1)

print(X.shape,Xtest.shape)

(20012, 370) (13342, 370)


In [38]:
# m.fit(X,y,eval_set=(X, y),early_stopping_rounds=100,verbose=100,eval_metric='auc')
# y_pred=cat_model.predict_proba(Xtest)[:,1]

In [39]:
# s=pd.DataFrame({'AGREEMENTID':test['AGREEMENTID'],'FORECLOSURE':y_pred})

# # s.to_csv('catboost1.csv',index=False) --main 62 max depth-8 lr-0.1
# # s.to_csv('catboostBasicnewbest.csv',index=False)
# s.to_csv('lgbmbasic.csv',index=False)
# s.head()

In [40]:

err=[]
y_pred_tot=[]
from sklearn.model_selection import KFold,StratifiedKFold
fold=StratifiedKFold(n_splits=30,shuffle=True,random_state=1994)
i=1
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
#     cat_model5 = CatBoostClassifier(n_estimators=1000,eval_metric='AUC',random_seed=1994,depth=6,bagging_temperature = 1.0)
#     cat_model5.fit(X_train.values,y_train.values,cat_features=categorical_features_indices,eval_set=(X_test, y_test),
#         plot=False,early_stopping_rounds=100,use_best_model=True,verbose=100) # early stopping set to 100 to prevent overfitting
#     preds = cat_model5.predict_proba(X_test)[:,1]
    
#     m=LGBMClassifier(n_estimators=1500,random_state=1994,learning_rate=0.03,max_depth=7,num_leaves=100,class_weight='balanced',reg_alpha=2)
    m=LGBMClassifier(n_estimators=1500,random_state=1994)
    # m=RidgeCV(cv=4)
    m.fit(X_train.values,y_train.values,eval_set=[(X_val, y_val.values)],eval_metric='auc', early_stopping_rounds=100,verbose=100)
    preds=m.predict_proba(X_test)[:,-1]
    
    print("err: ",roc_auc_score(y_test,preds))
    err.append(roc_auc_score(y_test,preds))
    p = m.predict_proba(Xtest)[:,-1]
#     s=pd.DataFrame({'AGREEMENTID':test['AGREEMENTID'],'FORECLOSURE':p})
#     s.to_csv('lgbmfoldsnew_xx'+str(i)+'.csv',index=False)
    i=i+1
    y_pred_tot.append(p)

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 1	valid_0's binary_logloss: 0.00194227
Early stopping, best iteration is:
[52]	valid_0's auc: 1	valid_0's binary_logloss: 0.008957
err:  0.9972565157750344
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.999796	valid_0's binary_logloss: 0.00323247
Early stopping, best iteration is:
[53]	valid_0's auc: 0.999877	valid_0's binary_logloss: 0.00971252
err:  0.9987997256515776
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.999999	valid_0's binary_logloss: 0.00222937
[200]	valid_0's auc: 1	valid_0's binary_logloss: 0.000328897
Early stopping, best iteration is:
[107]	valid_0's auc: 1	valid_0's binary_logloss: 0.00189413
err:  0.9977709190672154
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 1	valid_0's binary_logloss: 0.00206919
Early stopping, best iteration is:
[77]	valid_0's auc: 1	valid_0's bin

In [41]:
err_avg=np.mean(err,0)
err_avg

0.9977399795950737

In [42]:
y_pred=np.mean(y_pred_tot,0)
y_pred

array([9.94584885e-01, 9.88631636e-01, 9.96660139e-01, ...,
       3.89689534e-04, 6.32465975e-04, 4.97484427e-04])

In [43]:
# y_test_pred=(y_test_pred+y_pred)/2

In [44]:


s=pd.DataFrame({'AGREEMENTID':test['AGREEMENTID'],'FORECLOSURE':y_pred})

# s.to_csv('catboost1.csv',index=False) --main 62 max depth-8 lr-0.1
s.to_csv('lgbmbestfoldsxxXX.csv',index=False)
s.head()

,AGREEMENTID,FORECLOSURE
0,11220003,0.994585
1,11220004,0.988632
2,11220005,0.996660
3,11220007,0.993270
4,11220009,0.000784
